# ML-7. ПРОГНОЗИРОВАНИЕ БИОЛОГИЧЕСКОГО ОТВЕТА (HW-3)

In [34]:
# Импортируем необходимые библиотеки

import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [35]:
# Считаем данные

data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Создаем матрицу наблюдений X и вектор ответов y

X = data.drop(['Activity'], axis=1)
y = data['Activity']

# Разделяем выборку на тренировочную и тестовую в соотношении 80/20. 
# Для сохранения соотношений целевого признака используем параметр stratify.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state = 1, test_size = 0.2)


In [37]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=50)
#Обучаем модель
log_reg.fit(X_train, y_train)
#Выводим значения метрики
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


c:\Users\Home\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42, n_estimators=50)
#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.80


## Подбор гиперпараметров

### GridSearchCV

In [39]:
# Указываем искомые гиперпараметры
param_grid = {
    'penalty': ['l2', 'none'] ,
    'solver': ['lbfgs', 'sag'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}

# Вызваем объект класса GridSearchCV и передаем модель
grid_search_log_reg = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
# Обучаем модель
grid_search_log_reg.fit(X_train, y_train) 
y_test_pred = grid_search_log_reg.predict(X_test)
#Выводим значения метрики
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_log_reg.best_params_))


f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}


c:\Users\Home\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [40]:
# Указываем искомые гиперпараметры
param_grid = {'min_samples_leaf': list(range(1, 10, 2)),
              'max_depth': list(range(100, 1000, 200)),
              'criterion':['entropy','gini']
              }

# Вызваем объект класса GridSearchCV и передаем модель 
grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42, n_estimators=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
# Обучаем модель
grid_search_rf.fit(X_train, y_train) 
y_test_pred = grid_search_rf.predict(X_test)
#Выводим значения метрики
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rf.best_params_))


f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 3}


### RANDOMIZEDSEARCHCV 

In [41]:
# Указываем искомые гиперпараметры
param_distributions = {
    'penalty': ['l2', 'none'] ,
    'solver': ['lbfgs', 'sag'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}

# Вызваем объект класса RandomizedSearchCV и передаем модель             
random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_jobs = -1
)  
# Обучаем модель 
random_search_lr.fit(X_train, y_train) 
y_test_pred = random_search_lr.predict(X_test)
# Выводим значения метрики
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_lr.best_params_))

f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.45}


c:\Users\Home\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [42]:
# Указываем искомые гиперпараметры
param_distributions = {
    'min_samples_leaf': list(range(1, 10, 1)),
    'max_depth': list(range(10, 1000, 10)),
    'criterion':['entropy','gini']
}

# Вызваем объект класса RandomizedSearchCV и передаем модель            
random_search_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42, n_estimators=50),
    param_distributions=param_distributions, 
    cv=5, 
    n_jobs = -1,
    n_iter = 50
)  
# Обучаем модель
random_search_rf.fit(X_train, y_train) 
y_test_pred = random_search_rf.predict(X_test)
# Выводим значения метрики
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_rf.best_params_))

f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'min_samples_leaf': 2, 'max_depth': 720, 'criterion': 'entropy'}


### HYPEROPT

In [43]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 10, 1000, 10),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1)
      }

In [44]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    # model.fit(X, y)
    # score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [45]:
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [02:48<00:00,  8.42s/trial, best loss: -0.8098530424063289]
Наилучшие значения гиперпараметров {'max_depth': 300.0, 'min_samples_leaf': 2.0, 'n_estimators': 152.0}


In [46]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


f1_score на тестовом наборе: 0.82


### OPTUNA

In [47]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 1000, 10)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  #model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [48]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-08-17 20:36:04,252] A new study created in memory with name: RandomForestClassifier
[I 2022-08-17 20:36:12,861] Trial 0 finished with value: 0.8053533048180984 and parameters: {'n_estimators': 118, 'max_depth': 240, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8053533048180984.
[I 2022-08-17 20:36:23,057] Trial 1 finished with value: 0.8057127282663255 and parameters: {'n_estimators': 146, 'max_depth': 230, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8057127282663255.
[I 2022-08-17 20:36:29,965] Trial 2 finished with value: 0.8038937049340154 and parameters: {'n_estimators': 101, 'max_depth': 240, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.8057127282663255.
[I 2022-08-17 20:36:40,802] Trial 3 finished with value: 0.8107138943151775 and parameters: {'n_estimators': 137, 'max_depth': 920, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.8107138943151775.
[I 2022-08-17 20:36:49,145] Trial 4 finished with value: 0.8023482119991456 and parameters: 

Wall time: 3min 36s


In [49]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82
